# Converting traces in raw format

In this prerequisite part, we will first change the trace format, from the npy generated by the capture, to a "raw" format, simply consisting of writing the different values measured consecutively. Of course, the number of samples in one trace and the number of traces must be kept alongside the file (for example in a header file with #define).
First, let us make a capture.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'TINYAES128C'

In [2]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [3]:
fw_path = "../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)

In [4]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-aes-CWLITEXMEGA.hex
rm -f -- simpleserial-aes-CWLITEXMEGA.eep
rm -f -- simpleserial-aes-CWLITEXMEGA.cof
rm -f -- simpleserial-aes-CWLITEXMEGA.elf
rm -f -- simpleserial-aes-CWLITEXMEGA.map
rm -f -- simpleserial-aes-CWLITEXMEGA.sym
rm -f -- simpleserial-aes-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes.s aes-independant.s
rm -f -- simpleserial-aes.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes.d aes-independant.d
rm -f -- simpleserial-aes.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes.i aes-independant.i
.
-------- begin --------
avr-gcc (GCC) 4.9.2
Copyright (C) 2014 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simpleserial-aes.c
avr-gcc -c -mmcu=atxme

In [5]:
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 4043 bytes


In [28]:
# Capture Trace
import time
from tqdm import tnrange


ktp = cw.ktp.Basic()
traces = []
N = 1000 # the number of plaintext
nb_samples = 3000
scope.adc.samples = nb_samples
scope.adc.offset = 0

for i in tnrange(N, desc = 'Capturing traces'):
    key, text = ktp.next()  # creation of a pair comprising (fixed) key and text
    
    while True:
        trace = cw.capture_trace(scope, target, text, key) 
        if len(trace.wave) == nb_samples:
            break;
    
    if trace is None:
        continue
    
    traces.append(trace)
    

We will now print the first 10 values and write the trace to a file in "raw" format.

In [29]:
import numpy as np
import os

traces_dir = 'traces'
#os.mkdir(traces_dir)
trace_array = np.asarray([t for t in trace.wave for trace in traces])
"""
print first and last 10 elements

for i in range(10):
    print('%f' % trace_array[i])
print("...")
for i in range(len(trace_array)-10,len(trace_array)):
    print('%f' % trace_array[i])
"""

f = open(os.path.join(traces_dir, 'key.raw'), "wb")
f2 = open(os.path.join(traces_dir, 'plain.raw'), "wb")
f3 = open(os.path.join(traces_dir, 'traces.raw'), "wb")

# same key for all plaintext so write only once
print("key   : ",traces[0].key)
for b in traces[0].key:
        f.write(np.uint8(b))
        
for trace in traces:
    print("plain : ",trace.textin)
    for b in trace.textin:
        f2.write(np.uint8(b))
    for b in trace.wave:
        f3.write(b)
        
f.close()
f2.close()
f3.close()

key   :  CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
plain :  CWbytearray(b'bb 31 f1 98 c5 ca 68 19 4d b3 91 1b 76 22 08 3b')
plain :  CWbytearray(b'c4 dc a8 01 78 77 f7 3d 0e bd 32 c4 22 98 62 70')
plain :  CWbytearray(b'64 ee be da 1c 8d 2a 48 81 d1 2c 37 e8 7a 26 5c')
plain :  CWbytearray(b'98 31 a1 d0 58 2c c0 40 55 20 b7 b5 d5 91 ba 83')
plain :  CWbytearray(b'f4 c1 af b3 7c 8e c8 fe 6d 1a 4c 1c 69 d3 2a bc')
plain :  CWbytearray(b'35 e1 43 dd ee 56 9f 97 54 a8 df f8 5e e4 c5 28')
plain :  CWbytearray(b'11 80 84 40 8e 62 68 51 a1 6b fb d7 b1 71 bb eb')
plain :  CWbytearray(b'62 4b 94 34 f2 08 4a dd 59 94 27 e3 a4 3f 11 86')
plain :  CWbytearray(b'ac a0 eb f9 48 58 a3 36 58 a1 c9 c3 21 a0 1f 5c')
plain :  CWbytearray(b'b9 e2 58 dd 91 53 a4 82 be 17 05 7c 43 86 32 ee')
plain :  CWbytearray(b'59 c9 16 11 55 d0 f7 35 d8 b5 0c 3a c7 bc 7a 4b')
plain :  CWbytearray(b'84 9a fb de 25 1b c3 94 89 b4 89 c7 97 49 7f a3')
plain :  CWbytearray(b'18 26 63 80 2f 9a 4b 86 27 c

You must now write a C program which reads the file into an array and prints the first 10 values, using the following code skeleton, in which path is a buffer containing the file name.

    #define NB_SAMPLES 3000  // should be in a header file !
    
    double traces[NB_SAMPLES];  //only one trace here, to be adapted if needed
    
    if ((fd = open(path, O_RDONLY)) == -1) {
        perror("open");
        return -1;
    }

    if (read(fd, traces, sizeof(double) * NB_SAMPLES) == -1) {
        perror("read");
        return -1;
    }
    
    for (int i = 0; i < 10; i += 1) {
        printf("%f\n", traces[i]);
    }
    

Verify that you observe the same 10 values than in python. Change the python code above in order to convert the paintext and the key in raw type, using `astype('uint8')`, and write them in separate files.
You can check the content of the raw key file (for example) using either the linux commands `xxd` or `hexdump`.

Modify your script in order to generate raw files for a set of several traces with different random plaintexts. The file containing the traces and the file contaning the plaintext must have a length of `nb_samples * nb_traces * sizeof(double)` and `16 * nb_traces` respectively, which you can check with the linux `ls -l` command.

Note: it is possible that some traces contain an incorrect number of samples. It is advised to check the length of the trace right after its capture, and redo the capture if it is incorrect.

# Implementing CPA

In [27]:
import math
# !!! just to check that I have the same results in C and in python !!!

sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

HW = [bin(n).count("1") for n in range(0, 256)]


def intermediate(pt, key):
    return sbox[pt ^ key]


P = []

for i in range(N):
    line = [HW[intermediate(traces[i].textin[0],k)] for k in range(256)]
    P.append(line)
P = np.array(P)


print("\n")
print("P : ",end="")
print(P[0])

print("\n")
print("H_mean : ",end="")
for i in range(10):
    print(np.mean(P[:,i])," ",end="")

print("\n")
print("traces : ",end="")
for i in range(10):
    print(traces[i].wave[0]," ",end="")

print("\n")
print("T_mean : ",end="")
for i in range(10):
    print(np.mean([t.wave[i] for t in traces])," ",end="")    

    




P : [4 3 4 2 5 4 3 6 6 4 4 4 4 2 4 6 7 6 5 5 2 1 5 4 5 5 6 4 4 5 6 6 6 4 4 5 3
 2 1 4 2 4 2 4 1 5 3 4 4 4 3 3 3 4 5 5 4 6 7 4 6 7 4 3 3 3 3 6 4 5 6 4 1 6
 4 5 4 4 0 6 3 5 5 2 3 5 5 5 4 5 4 2 2 3 3 3 1 8 6 4 5 5 5 2 3 5 3 6 3 4 1
 5 2 4 6 3 3 6 1 7 3 4 4 3 3 7 4 7 6 5 3 6 3 6 4 2 2 3 2 2 2 2 5 5 3 5 3 5
 3 5 6 5 6 5 2 4 5 2 3 5 4 5 5 1 4 4 5 5 4 5 4 4 6 3 5 5 3 4 4 5 3 5 4 3 3
 2 2 4 3 3 4 2 3 3 4 5 3 4 5 5 2 2 6 3 3 5 5 4 4 6 4 3 4 6 4 5 3 4 4 4 5 4
 3 4 3 3 6 3 2 4 4 4 7 5 2 3 3 3 3 3 5 4 2 7 5 4 4 5 4 5 4 3 4 5 3 2]


H_mean : 3.988  3.997  4.0136666666666665  3.969333333333333  4.005333333333334  3.9963333333333333  3.985333333333333  4.012333333333333  4.009333333333333  4.0103333333333335  

traces : 0.0654296875  0.0634765625  0.0498046875  0.06640625  0.0634765625  0.0615234375  0.06640625  0.0498046875  0.0615234375  0.05859375  

T_mean : 0.0613310546875  -0.2962666015625  -0.13730143229166666  -0.13683658854166666  0.014869140625  -0.3881806640625  -0.2040693359375  -0.1867

You must now implement a CPA (Correlation Power Analysis) in C, using traces that you have converted as above. It should be much faster to execute in C than in python. You must use the Pearson correlation coefficient as presented during the lectures. 

Your code must be as generic as possible regarding the number of samples, traces, and file names. It should allow to perform the attack on a sub-interval of the traces, by reading into memory only this sub-interval for each trace.

In your report, you must present the results of an evaluation regarding the effect of the trace number on the success rate of the attack (in terms of number of key bytes correctly recovered), and compare it with the DPA results.

Finally, as a bonus, you can implement the DPA attack in C in order to compare the execution time between both attacks, and between the C and the python implementation. Explain the results that you have obtained in your report.